# A simple bijective network for data transformation using strings

A bijective data tranformation network is a series of nodes that represent different data formats with the same content. Nodes are connected by edges that are functions that are unidirectional mappings from one node to another. A graph is formed by showing that nodes that are transformed through a series of other nodes return to their original state. This requires that 
1. a node is well defined (has a name)
2. a node has an equallity operator n1==n1'
3. Has at least 1 edge that enters the node
4. Has at least 1 edge that leaves the nod

The purpose of this network is to create a way of transforming data from one format to another as efficcently as possible while preserving contnent. As an example we use a series of data types that are strings.

In [8]:
# the edge from list of strings to string is defined in General Models
from pyMeasure.Code.DataHandlers.GeneralModels import *
# node 1
node_1="This is a test string\n it has to have multiple lines \n and many characters 34%6\n^"
# edge that maps node 1 to 2  
node_2=node_1.splitlines()
# edge that maps node 2 to 1
node_1_prime=string_list_collapse(node_2)
# make sure the round trip has not changed anything 
print("The assertion that n1==n1' is {0}").format(node_1==node_1_prime)
# we can add a file that is another node
# edge that maps node 1 to node 3
node_3=open("node_3.txt","w")
node_3.write(node_1)
node_3.close()
# now we can add an edge that maps node_3 to node_2
node_2_prime=[]
node_3=open("node_3.txt","r")
for line in node_3:
    node_2_prime.append(line.replace("\n",""))
node_3.close()
print("The assertion that n2==n2' is {0}").format(node_2==node_2_prime)

# now we can add a fourth node using the edge that maps node 2 to 4
node_4=[line+"\n" for line in node_2]
node_4[-1]=node_4[-1].replace("\n","")
node_1_double_prime=string_list_collapse(node_4,string_delimiter="")

print("The assertion that n1==n1'' is {0}").format(node_1==node_1_double_prime)

The assertion that n1==n1' is True
The assertion that n2==n2' is True
The assertion that n1==n1'' is True


## Now we define the graph class

In [211]:
import re
import datetime
import numpy as np
class Graph():
    def __init__(self):
        """Initializes the graph. The first 2 nodes and two edges forming a bijection between them are required"""
        self.node_names=['n1','n2']
        self.current_node='n1'
        self.state=[1,0]
        self.edges=[]
        self.edge_matrices=[]
        self.data="This is a test string\n it has to have multiple lines \n and many characters 34%6\n^"
        self.state_matrix=np.matrix(self.state).T
        
    def add_edge(self,begin_node=None,end_node=None,edge_function=None):
        """Adds an edge mapping one node to another, required input is begin_node (it's name)
        end_node, and the edge function"""
        # check to see if edge is defined if it is increment a number
        edge_match=re.compile("edge_{0}_{1}".format(begin_node,end_node))
        keys=self.__dict__.keys()
        #print keys
        iterator=0
        for key in keys:
            if re.match(edge_match,key):
                iterator+=1
        edge_name="edge_{0}_{1}_{2:0>3d}".format(begin_node,end_node,iterator)
        self.__dict__[edge_name]=edge_function
        self.edges.append(edge_name)
        edge_matrix=np.zeros((len(self.state),len(self.state)))
        begin_position=self.node_names.index(begin_node)
        end_position=self.node_names.index(end_node)
        edge_matrix[end_position][begin_position]=1
        edge_matrix=np.matrix(edge_matrix)
        self.edge_matrices.append(edge_matrix)
        
    def move_to(self,path):
        """Changes the state of the graph by moving along the path specified"""
        #print path
        for index,edge in enumerate(path):
            #print edge
            edge_pattern='edge_(?P<begin_node>\w+)_(?P<end_node>\w+)_(?P<iterator>\w+)'
            match=re.match(edge_pattern,edge)
            begin_node=match.groupdict()['begin_node']
            end_node=match.groupdict()['end_node']
            print("moving {0} -> {1}".format(begin_node,end_node))
            #print self.data
            self.data=self.__dict__[edge](self.data)
            #print self.data
            self.current_node=match.groupdict()['end_node']
            self.state=[0 for i in range(len(self.node_names))]
            position=self.node_names.index(self.current_node)
            self.state[position]=1
            self.state_matrix=np.matrix(self.state).T
            #print self.state
            #print self.current_node
            
    def virtual_move_to(self,path):
        """Virtual move to simulates moving but does not change the state of the graph"""
        #print path
        temp_state=self.state
        temp_data=self.data
        temp_current_node=self.current_node
        temp_node_names=self.node_names
        for index,edge in enumerate(path):
            #print edge
            edge_pattern='edge_(?P<begin_node>\w+)_(?P<end_node>\w+)_(?P<iterator>\w+)'
            match=re.match(edge_pattern,edge)
            begin_node=match.groupdict()['begin_node']
            end_node=match.groupdict()['end_node']
            #print("moving {0} -> {1}".format(begin_node,end_node))
            #print self.data
            temp_data=self.__dict__[edge](temp_data)
            #print self.data
            temp_current_node=match.groupdict()['end_node']
            temp_state=[0 for i in range(len(temp_node_names))]
            position=temp_node_names.index(temp_current_node)
            temp_state[position]=1
            #print temp_state
            #print self.state
            #print self.current_node  
            
    def __str__(self):
        return str(self.data)
    
    def add_node(self,node_name,edge_into_node_begin,edge_into_node_function,edge_out_node_end,edge_out_node_function):
        """Adds a node to the graph. Required input is node_name (a string with no spaces), a reference to an entering node,
        the function mapping the entering node to the new node, a reference to an exiting node and the function mapping the
        new node to the exiting node."""
        # first check if node into and out of node is good
        self.node_names.append(node_name)
        self.state.append(0)
        self.state_matrix=np.matrix(self.state).T
        for index,matrix in enumerate(self.edge_matrices):
            pad_row=np.zeros((1,len(matrix)))
            new_matrix=np.concatenate((matrix, pad_row), axis=0)
            pad_column=np.zeros((1,len(self.node_names)))
            new_matrix=np.concatenate((new_matrix, pad_column.T), axis=1)
            #print("New matrix is :\n{0}".format(new_matrix))
            self.edge_matrices[index]=new_matrix
        self.add_edge(begin_node=node_name,end_node=edge_out_node_end,edge_function=edge_out_node_function)
        self.add_edge(begin_node=edge_into_node_begin,end_node=node_name,edge_function=edge_into_node_function)
    
    def path_length(self,path,num_repeats=10):
        """Determines the length of a given path, currently the metric is based on the time to move to."""
        begin_time=datetime.datetime.now()
        #num_repeats=100
        for i in range(num_repeats):
            self.virtual_move_to(path)
        end_time=datetime.datetime.now()
        delta_t=end_time-begin_time
        path_length=delta_t.total_seconds()/float(num_repeats)
        if path_length ==0.0:
            print("Warning the path length is less than 1 microsecond, make sure num_repeats is high enough to measure it.")
        return path_length
                
    def is_path_valid(self,path):
        """Returns True if the path is valid from the current node position or False otherwise"""
        null_state=[0 for i in range(len(self.node_names))]
        null_state_matrix=np.matrix(null_state).T
        new_state=np.matrix(self.state).T
        for index,edge in enumerate(path):
            #print index
            #print edge
            edge_position=self.edges.index(edge)
            move_matrix=self.edge_matrices[edge_position]
            #print move_matrix
            new_state=move_matrix*new_state
            if new_state.any()==null_state_matrix.any():
                #print new_state
                #print null_state_matrix
                return False
        return True
            
            
    def get_entering_nodes(self,node): 
        """Returns all nodes that have an edge that enter the specificed node"""
        enter_edge_pattern=re.compile('edge_(?P<begin_node>\w+)_{0}_(?P<iterator>\w+)'.format(node))
        enter_nodes=[]
        for index,edge in enumerate(self.edges):
            enter_match=re.match(enter_edge_pattern,edge)
            if enter_match:
                enter_node=enter_match.groupdict()['begin_node']
                enter_nodes.append(enter_node)
        return enter_nodes
    
    def get_entering_edges(self,node): 
        """Returns all edges that enter the specificed node"""
        enter_edge_pattern=re.compile('edge_(?P<begin_node>\w+)_{0}_(?P<iterator>\w+)'.format(node))
        enter_edges=[]
        for index,edge in enumerate(self.edges):
            if re.match(enter_edge_pattern,edge):
                enter_edges.append(edge)
        return enter_edges
    
    def get_exiting_edges(self,node):
        """Returns all edges that exit the specificed node"""
        exit_edge_pattern=re.compile('edge_{0}_(?P<end_node>\w+)_(?P<iterator>\w+)'.format(node))
        exit_edges=[]
        for index,edge in enumerate(self.edges):
            if re.match(exit_edge_pattern,edge):
                exit_edges.append(edge)
        return exit_edges 
    
    def get_exiting_nodes(self,node): 
        """Returns all nodes that have an edge leaving the specificed node"""
        exit_edge_pattern=re.compile('edge_{0}_(?P<end_node>\w+)_(?P<iterator>\w+)'.format(node))
        exit_nodes=[]
        for index,edge in enumerate(self.edges):
            exit_match=re.match(exit_edge_pattern,edge)
            if exit_match:
                exit_node=exit_match.groupdict()['end_node']
                exit_nodes.append(exit_node)
        return exit_nodes
    
    def get_path(self,first_node,last_node):
        """Returns the first path found between first node and last node"""
        edge_pattern=re.compile('edge_(?P<begin_node>\w+)_(?P<end_node>\w+)_(?P<iterator>\w+)')
        exit_paths=new_graph.get_exiting_edges(first_node)
        next_nodes=new_graph.get_exiting_nodes(first_node)
        possible_paths=[]
        for exit_path in exit_paths:
            possible_paths.append([exit_path])
        #print possible_paths
        for i in range(len(new_graph.node_names)):
            for index,path in enumerate(possible_paths):
                last_edge=path[-1]
                match=re.match(edge_pattern,last_edge)
                end_node=match.groupdict()['end_node']
                #print next_node
                if end_node==last_node:
                    #print("The path found is {0}".format(path))
                    return path
                next_possible_paths=[]
                next_path=path
                next_edges=new_graph.get_exiting_edges(end_node)
                next_nodes=new_graph.get_exiting_nodes(end_node)
                for index,edge in enumerate(next_edges):
                    next_node=next_nodes[index]
                    #print next_node
                    next_path.append(edge)
                    #print next_path
                    if next_node==last_node:
                        #print("The path found is {0}".format(next_path))
                        return next_path
                    next_possible_paths.append(next_path)
                possible_paths=next_possible_paths             

        
        
        
def edge_1_to_2(in_string):
    return in_string.splitlines()
    
def edge_2_to_1(string_list):
    return string_list_collapse(string_list)

def edge_1_to_3(in_string):
    node_3=open("node_3.txt","w")
    node_3.write(in_string)
    node_3.close()
    return "node_3.txt"
    
def edge_3_to_2(file_name):
    node_2_prime=[]
    node_3=open(file_name,"r")
    for line in node_3:
        node_2_prime.append(line.replace("\n",""))
    node_3.close()
    return node_2_prime
    
new_graph=Graph()
new_graph.add_edge('n1','n2',edge_1_to_2)
new_graph.add_edge('n2','n1',edge_2_to_1)
#print new_graph.__dict__
#new_graph.move_to(*new_graph.edges)
#print new_graph.edge_n1_n2_000(node_1)
#print new_graph.__dict__
new_graph.add_node('n3','n1',edge_1_to_3,'n2',edge_3_to_2)
#print new_graph.data
#print new_graph.__dict__['edge_n1_n2_000'](new_graph.data)
#print new_graph.edges
new_path=['edge_n1_n2_000', 'edge_n2_n1_000', 'edge_n1_n3_000', 'edge_n3_n2_000','edge_n2_n1_000']
path_length_1=new_graph.path_length(new_path)
new_path=['edge_n1_n2_000']
path_length_2=new_graph.path_length(new_path)
print("The path length of the first path length is {0:.9f} and the second path length is {1:.9f}".format(path_length_1,path_length_2))
#print new_graph.__dict__
print new_graph.get_path('n2','n2')
path=new_graph.get_path('n1','n1')
print path
print new_graph.is_path_valid(path)

Warning the path length is less than 1 microsecond, make sure num_repeats is high enough to measure it.
The path length of the first path length is 0.000900000 and the second path length is 0.000000000
['edge_n2_n1_000', 'edge_n1_n2_000']
['edge_n1_n2_000', 'edge_n2_n1_000']
True


In [208]:
first_node='n3'
last_node='n2'
edge_pattern=re.compile('edge_(?P<begin_node>\w+)_(?P<end_node>\w+)_(?P<iterator>\w+)')
exit_paths=new_graph.get_exiting_edges(first_node)
next_nodes=new_graph.get_exiting_nodes(first_node)
possible_paths=[]
for exit_path in exit_paths:
    possible_paths.append([exit_path])
#print possible_paths
for i in range(len(new_graph.node_names)):
    for index,path in enumerate(possible_paths):
        last_edge=path[-1]
        match=re.match(edge_pattern,last_edge)
        end_node=match.groupdict()['end_node']
        
        #print next_node
        if end_node==last_node:
            print("The path found is {0}".format(path))
        next_possible_paths=[]
        next_path=path
        next_edges=new_graph.get_exiting_edges(end_node)
        next_nodes=new_graph.get_exiting_nodes(end_node)
        for index,edge in enumerate(next_edges):
            next_node=next_nodes[index]
            #print next_node
            next_path.append(edge)
            #print next_path
            if next_node==last_node:
                print("The path found is {0}".format(next_path))
            next_possible_paths.append(next_path)
        possible_paths=next_possible_paths


print possible_paths
print node_paths

The path found is ['edge_n3_n2_000']
The path found is ['edge_n3_n2_000', 'edge_n2_n1_000', 'edge_n1_n2_000']
The path found is ['edge_n3_n2_000', 'edge_n2_n1_000', 'edge_n1_n2_000', 'edge_n1_n3_000', 'edge_n3_n2_000']
The path found is ['edge_n3_n2_000', 'edge_n2_n1_000', 'edge_n1_n2_000', 'edge_n1_n3_000', 'edge_n3_n2_000']
[['edge_n3_n2_000', 'edge_n2_n1_000', 'edge_n1_n2_000', 'edge_n1_n3_000', 'edge_n3_n2_000', 'edge_n2_n1_000']]
[]


In [93]:
import numpy as np
state=np.matrix([0,1,0,0])

In [94]:
move_matrix=np.zeros((4,4))
move_matrix[0][1]=1
move_matrix=np.matrix(move_matrix)
print move_matrix

[[ 0.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [97]:
(move_matrix*state.T).T

matrix([[ 1.,  0.,  0.,  0.]])

In [98]:
move_matrix_2=np.zeros((4,4))
move_matrix_2[1][0]=1
move_matrix_2=np.matrix(move_matrix_2)

In [99]:
move_matrix_2*move_matrix

matrix([[ 0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]])

In [112]:
print len(move_matrix)
pad=np.zeros((1,len(move_matrix)))

4


In [113]:
pad

array([[ 0.,  0.,  0.,  0.]])

In [115]:
padded=np.concatenate((move_matrix, pad), axis=0)


In [116]:
padded

matrix([[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]])

In [129]:
test_list=[1,2,2,2,3]
test_list.index(2)

1